In [1]:
# imports
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
import copy

In [2]:
class Config:
    machine_data_path = Path('./machine.csv')
    titanic_data_path = Path('./Titanic.csv')

In [3]:
class HelperFunctions:
    @staticmethod
    def mean(x):
        n = x.shape[0]
        return x.sum() / n
    
    @staticmethod
    def std(x):
        n = x.shape[0]
        x_mean = HelperFunctions.mean(x)
        return np.sqrt(1/n*(((x-x_mean)**2).sum()))
    
    @staticmethod
    def covariance(x, y):
        n = x.shape[0]
        xy_mean = np.multiply(x, y).sum() / n
        x_mean = HelperFunctions.mean(x)
        y_mean = HelperFunctions.mean(y)
        return xy_mean - x_mean*y_mean
    
    @staticmethod
    def correlation(x, y):
        corr = 0
        corr += HelperFunctions.covariance(x, y) 
        corr /= HelperFunctions.std(x)
        corr /= HelperFunctions.std(y)
        return corr
    
    @staticmethod
    def correlation_matrix(x):
        n = x.shape[1]
        cm = [[0 for i in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                cm[i][j] = HelperFunctions.correlation(x[:, i], x[:, j]) 
        return cm
    
    @staticmethod
    def conv_sum(x, y):
        return (x*y).sum(axis=1)

# Cross validation

In [4]:
titanic_df = pd.read_csv(Config.titanic_data_path)
titanic_df.head()

,Class,Age,Sex,Survived
0,First,Adult,Male,Yes
1,First,Adult,Male,Yes
2,First,Adult,Male,Yes
3,First,Adult,Male,Yes
4,First,Adult,Male,Yes


In [5]:
train_df, valid_df = train_test_split(titanic_df, 
                                    test_size=0.2,
                                    shuffle=True)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

# Decision Tree with GINI index

In [6]:
class DecisionTreeNode:
    def __init__(self, curr_column, nxt, leaf_node, pred):
        self.curr_column = curr_column
        self.nxt = nxt
        self.leaf_node = leaf_node
        self.pred = pred

# using gini index
class DecisionTree:
    def __init__(self, df, targ_column):
        self.df = df
        self.feat_cnt = len(df.columns) - 1
        self.targ_column = targ_column

    def eval_gini_index(self, d):
        cntr = Counter(d)
        total = 0
        for i in cntr.values():
            total += i
        gini_index = 1
        for i in cntr.values():
            p = i/total
            gini_index -= (p**2)
        return gini_index
    
    def build(self, max_depth=1000000007):
        considered = dict()
        for i in self.df.columns:
            if i != self.targ_column:
                considered[i] = False
        root = self.dfs(self.df, considered, 1, max_depth)
        return root
        
    def dfs(self, df, considered, depth, max_depth):
        mx_gain = 0.0
        mx_col = None
        # gini index of the passed df
        s = self.eval_gini_index(df[self.targ_column])
        cnt = len(df)
        for col in df.columns:
            if col!=self.targ_column and not considered[col]:
                gini_index_col = 0
                for val in df[col].unique():
                    val_df = df[df[col]==val]
                    p =  len(val_df) / cnt
                    gini_index_col += p*self.eval_gini_index(val_df[self.targ_column])
                # gini index gain
                g = s - gini_index_col
                if g >= mx_gain:
                    mx_gain = g
                    mx_col = col
        tn = None
        nxt = dict()
        # leaf node conditions
        if mx_col is None or mx_gain == 0.0 or depth>max_depth:
            mx_occr_targ = df[self.targ_column].mode()[0]
            tn = DecisionTreeNode(None, nxt, True, mx_occr_targ)
        else:
            considered[mx_col] = True
            for i in df[mx_col].unique():
                nxt[i] = self.dfs(df[df[mx_col]==i], considered, depth+1, max_depth)
            tn = DecisionTreeNode(mx_col, nxt, False, None)
            considered[mx_col] = False
        return tn

In [7]:
def display_tree(root, depth=0):
    s = '\t'*depth
    if root.leaf_node:
        print(f'{s}Prediction: {root.pred}')
        print()
        return
    for i in root.nxt.items():
        print(f'{s}{root.curr_column}', end=' ')
        print(f'= {i[0]}')
        display_tree(i[1], depth+1)

In [8]:
dt = DecisionTree(train_df, 'Survived')
root = dt.build()

In [9]:
display_tree(root)

Sex = Male
	Class = Third
		Age = Adult
			Prediction: No

		Age = Child
			Prediction: No

	Class = Crew
		Prediction: No

	Class = First
		Age = Adult
			Prediction: No

		Age = Child
			Prediction: Yes

	Class = Second
		Age = Adult
			Prediction: No

		Age = Child
			Prediction: Yes

Sex = Female
	Class = Third
		Age = Adult
			Prediction: No

		Age = Child
			Prediction: No

	Class = Second
		Age = Adult
			Prediction: Yes

		Age = Child
			Prediction: Yes

	Class = First
		Age = Adult
			Prediction: Yes

		Age = Child
			Prediction: Yes

	Class = Crew
		Prediction: Yes



# Checking for overfitting

In [10]:
def predict(root, df):
    preds = []
    for idx, row in df.iterrows():
        curr_root = root
        while not curr_root.pred:
            curr_col = curr_root.curr_column
            curr_root = curr_root.nxt[row[curr_col]]
            continue
        preds.append(curr_root.pred)
    return np.array(preds)

In [11]:
train_preds = predict(root, train_df)
valid_preds = predict(root, valid_df)

In [12]:
def accuracy(preds, targ):
    n = preds.shape[0]
    return (preds==targ).sum() / n

In [13]:
train_acc = accuracy(train_preds, train_df['Survived'])
valid_acc = accuracy(valid_preds, valid_df['Survived'])

In [14]:
print(f'For the full decision tree:')
print(f'Training Accuracy: {train_acc}\nValidation Accuracy: {valid_acc}')

For the full decision tree:
Training Accuracy: 0.7846590909090909
Validation Accuracy: 0.8140589569160998


Since the training and validation accuracy are so close we are not overfitting

# Using pre-prunning

In [15]:
preprunned_dt = DecisionTree(train_df, 'Survived')
preprunned_root = preprunned_dt.build(max_depth=2)

In [16]:
display_tree(preprunned_root)

Sex = Male
	Class = Third
		Prediction: No

	Class = Crew
		Prediction: No

	Class = First
		Prediction: No

	Class = Second
		Prediction: No

Sex = Female
	Class = Third
		Prediction: No

	Class = Second
		Prediction: Yes

	Class = First
		Prediction: Yes

	Class = Crew
		Prediction: Yes



In [17]:
train_preds = predict(preprunned_root, train_df)
valid_preds = predict(preprunned_root, valid_df)

In [18]:
train_acc = accuracy(train_preds, train_df['Survived'])
valid_acc = accuracy(valid_preds, valid_df['Survived'])

In [19]:
print('After preprunning:')
print(f'Training Accuracy: {train_acc}\nValidation Accuracy: {valid_acc}')

After preprunning:
Training Accuracy: 0.7767045454545455
Validation Accuracy: 0.8095238095238095


This model is generalizing much better

# Using post-prunning

In [20]:
def dt_postprune(root, parent, col_selected, full_dt, train_df, valid_df):
    if root.pred: 
        return root 
    cols = list()
    for col in root.nxt.keys(): cols.append(col)
    for col in cols:
        root.nxt[col] = dt_postprune(root.nxt[col], 
                            root,
                            col,
                            full_dt,
                            train_df[train_df[root.curr_column]==col], 
                            valid_df)
    if parent != -1:
        full_dt_preds = predict(full_dt, valid_df)
        acc = accuracy(full_dt_preds, valid_df['Survived'])

        mx_occr_targ = train_df['Survived'].mode()[0]
        prunned_node = DecisionTreeNode(None, dict(), True, mx_occr_targ)
        parent.nxt[col_selected] = prunned_node
        prunned_preds = predict(full_dt, valid_df)
        prunned_acc = accuracy(prunned_preds, valid_df['Survived'])

        if prunned_acc >= acc:
            return prunned_node
    return root

In [21]:
postprunned_root = dt_postprune(root, -1, -1, root, train_df, valid_df)
display_tree(postprunned_root)

Sex = Male
	Class = Third
		Prediction: No

	Class = Crew
		Prediction: No

	Class = First
		Age = Adult
			Prediction: No

		Age = Child
			Prediction: Yes

	Class = Second
		Age = Adult
			Prediction: No

		Age = Child
			Prediction: Yes

Sex = Female
	Class = Third
		Prediction: No

	Class = Second
		Prediction: Yes

	Class = First
		Prediction: Yes

	Class = Crew
		Prediction: Yes



In [22]:
train_preds = predict(postprunned_root, train_df)
valid_preds = predict(postprunned_root, valid_df)

In [23]:
train_acc = accuracy(train_preds, train_df['Survived'])
valid_acc = accuracy(valid_preds, valid_df['Survived'])

In [24]:
print('After post prunning:')
print(f'Training Accuracy: {train_acc}\nValidation Accuracy: {valid_acc}')

After post prunning:
Training Accuracy: 0.7846590909090909
Validation Accuracy: 0.8140589569160998
